In [4]:
import spacy
import textacy

print('loading en_coref_md...')
nlp = spacy.load('en_coref_md')
print('...done')



# for now assuming all names are unique identifiers
class Person:
    statements = []

    def __init__(self, name, pronouns=None, refs=[], user=False):
        self.name = name
#         self.gender = gender
        self.refs = refs
        self.user = user




# UPGRADE AT SOME POINT TO EXTRACT GENDER, ACCOUNT FOR CLUSTERS WITHOUT NAMES
# UPGRADE TO INCLUDE I, USER

# assumes names are unique identifiers
# assumes misspellings are diff people

# MEMORYLESS FOR NOW; each change to text means a whole new model
# Set extensions later, for keeping track of which tokens are what
class Model:
    text = None
    doc = None
    people = []
    resolved_text = None
    resolved_doc = None

    def __init__(self, text):
        self.text = text
        self.doc = self.get_doc()
        self.people = self.get_people()
        self.resolved_text = self.get_resolved()
        self.resolved_doc = self.get_doc(self.resolved_text)

    def get_doc(self, text=None):
        if text == None:
            text = self.text
        preprocessed = textacy.preprocess.normalize_whitespace(text)
        preprocessed = textacy.preprocess.preprocess_text(preprocessed, fix_unicode=True, no_contractions=True, no_accents=True)
        doc = nlp(preprocessed)

        # merge mentions into tokens for easy coref tracking, resolution
        for cluster in doc._.coref_clusters:
            for mention in cluster.mentions:
                mention.merge()
        return doc

        def get_person_by_name(self, name):
            for person in self.people:
                if person.name == name:
                    return person
                    return None

    def get_people(self, doc=None):
        if doc == None:
            doc = self.doc
        namedrops = [ent for ent in doc.ents if ent.label_ == 'PERSON']
        names = set([namedrop.text for namedrop in namedrops])
        people = []

        # for clusters that include namedrops
        for cluster in doc._.coref_clusters:
            name = None

            for mention in cluster.mentions:
                mention_text = mention.root.text
                if mention_text in names:
                    name = mention_text

            if name != None:
                person = self.get_person_by_name(name)
                refs = [mention.head.i for mention in cluster.mentions]
                if person == None:
                    person = Person(name, refs=refs)
                    people += [person]
                else:
                    person.refs += refs

            # for named entities without clusters (single mentions)
            for name_mention in name_mentions:
                person = self.get_person_by_name(name_mention.text)
                if person == None:
                    person = Person(name_mention.text, refs=[name_mention.head.i])
                    people += [person]
        return people

    def get_resolved(self, doc=None):
        if doc == None:
            doc = self.doc
        token_texts = [token.text for token in doc]

        for person in self.people:
            for ref in person.refs:
                resolved = person.name
                if doc[ref].pos_ == 'ADJ':
                    resolved += '\'s'

                token_texts[ref] = resolved
        return ' '.join(token_texts)

    # def update_people_statements(self, doc):
    #     res = nlp(self.resolve_people(doc))
    #
    #     for person in model.people:
    #         statements = []
    #         for ref in person.refss:
    #             head = ref.root.head
    #             if head.pos_ == 'VERB':
    #                 for statement in textacy.extract.semistructured_statements(res, person.name, head.lemma_):
    #                     statements += [statement]
    #         person.statements = list(set(statements))


loading en_coref_md...
...done


In [5]:
text = "Sheila was run over by a truck. She herself didn't see that coming. I told her she should take care of herself, but I know she'll just go and do her thing regardless of what I say. What a conundrum! This makes me wish I had never signed up to be friends with her, although I do love the girl."

In [6]:
model = Model(text)

IndexError: [E037] Error calculating span: Can't find a token ending at character offset 35.

In [7]:
text = 'November was a trying month… on the 7th Dante had a major accident. 5 minutes before school and he and some friends are climbing the fence, I tell him it’s not a good idea and to get down. I turn back to talk to Jodi (on of my best mom friend’s at the school) and Dante comes to me screaming with his hand full of blood. I run him into my classroom and get him to the sink, as I turn on the water to clean the area the flap of his thumb lifts away and I see the bone. Shit. This isn’t something I can fix here, I grab my first aid kit and wrap it like crazy because it’s bleeding like crazy. I phone James and tell him to get to the ER as Dante is screaming and freaking out in the background as I’m trying to usher him back to the car as he’s bleeding like a stuffed pig. Unfortunately in the ER I learned that my child doesn’t take to freezing, an hour of gel freezing and he still felt the 2 needles as they went in, 15 minutes later and he felt the last 2 stitches of 8. He needed more because his finger still had gaps, the doctor didn’t want to cause him anymore pain so he glued them. It was an intense and deep gash that spiraled all the way up his thumb. I was trying to stay strong for him but I did break down as he screamed and cried, I was left to emotionally drained that day. James was able to take the remainder of the day off and stay with him. He missed 2 more days of school and then had an extra long weekend due to the holiday and the pro day but for 2 weeks he couldn’t write (of course it was his right hand.) 3 doctor visits later and he finally got them out full last week, the first visit the doctor wanted them in longer because of the severity. 2nd time he could only get 6 out because the glue had gotten on the last 2 stitches and he didn’t want to have to dig them out so we had to soak and dissolve the glue for 3 days. 3rd time the last 2 came out.  Even now he’s slowly regaining his writing skills as there was some nerve damage.'

In [9]:
adoc = nlp(text)
# for cluster in adoc._.coref_clusters:
#     for mention in cluster.mentions:
#         mention.merge()
        
for cluster in adoc._.coref_clusters:
    for mention in cluster.mentions:
        print(cluster)

the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
school: [school, the school]
school: [school, the school]
my first aid kit: [my first aid kit, it, it]
my first aid kit: [my first aid kit, it, it]
my first aid kit: [my first aid kit, it, it]


IndexError: [E037] Error calculating span: Can't find a token ending at character offset 1001.

In [10]:
adoc = nlp(text)
# for cluster in adoc._.coref_clusters:
#     for mention in cluster.mentions:
#         mention.merge()
        
for cluster in adoc._.coref_clusters:
    for mention in cluster.mentions:
        print(cluster)

the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
the 7th Dante: [the 7th Dante, he, him, Dante, his, him, him, his]
school: [school, the school]
school: [school, the school]
my first aid kit: [my first aid kit, it, it]
my first aid kit: [my first aid kit, it, it]
my first aid kit: [my first aid kit, it, it]
James: [James, him, him, he, he, he, He, his, him, he, his, him, he]
James: [James, him, him, he, he, he, He, his, him, he, his, him, he]
James: [James, him, him, he, he, he, He, his, him, he, his, him, he]
James: [James, him, him, he, he, he, He, his, him, he, his, him,